In [6]:
# -*- coding: utf-8 -*-
import numpy as np
from re import sub, findall, MULTILINE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from unicodedata import normalize
from json import loads
from csv import reader,writer,QUOTE_ALL
from sklearn.model_selection import train_test_split
from string import punctuation

from pickle import load
from operator import itemgetter
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from sklearn import svm, linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

#IMPORTING EXTERNAL JUPYTER NOTEBOOKS
# import nbimporter
# from readability import *
# import nltk_linguistic_features as nltk_linguis_features
# import utils as utils
# from importlib import reload
# import sys
# reload(sys.modules['readability'])
# reload(sys.modules['nltk_linguistic_features'])
# reload(sys.modules['utils'])


In [7]:
#######--------------------DICTIONARY OF FEATURES--------------------#######
# Summary:
# h_basic_filter = [0:3]
# h_webmarkup_filter = [4:17]
# h_liwc_filter = [18:110]
# h_readability_filter = [111:129]
# h_nlkt_filter = [130:162]

#Details
# features = {0:'urls', 1:'headline', 2:'content or headline+content', 3:'date', 
            
#             4:'author', 5:'freq_basic', 6:'freq_formatting', 7:'freq_forms_inputs', 8:'freq_frames', 9:'freq_images', 
#             10:'freq_audio_video', 11:'freq_links', 12:'freq_lists', 13:'freq_tables', 14:'freq_styles_semantics', 
#             15:'freq_meta_info', 16:'freq_programming_info', 17:'freq_ads', 
            
#             18:'WC', 19:'Analytic', 20:'Clout', 21:'Authentic', 22:'Tone', 23:'WPS', 24:'Sixltr', 25:'Dic', 
#             26:'function', 27:'pronoun', 28:'ppron', 29:'i', 30:'we', 31:'you', 32:'shehe', 33:'they', 34:'ipron', 
#             35:'article', 36:'prep', 37:'auxverb', 38:'adverb', 39:'conj', 40:'negate', 41:'verb', 42:'adj', 
#             43:'compare', 44:'interrog', 45:'number', 46:'quant', 47:'affect', 48:'posemo', 49:'negemo', 50:'anx', 
#             51:'anger', 52:'sad', 53:'social', 54:'family', 55:'friend', 56:'female', 57:'male', 58:'cogproc', 
#             59:'insight', 60:'cause', 61:'discrep', 62:'tentat', 63:'certain', 64:'differ', 65:'percept', 66:'see', 
#             67:'hear', 68:'feel', 69:'bio', 70:'body', 71:'health', 72:'sexual', 73:'ingest', 74:'drives', 
#             75:'affiliation', 76:'achieve', 77:'power', 78:'reward', 79:'risk', 80:'focuspast', 81:'focuspresent', 
#             82:'focusfuture', 83:'relativ', 84:'motion', 85:'space', 86:'time', 87:'work', 88:'leisure', 89:'home', 
#             90:'money', 91:'relig', 92:'death', 93:'informal', 94:'swear', 95:'netspeak', 96:'assent', 97:'nonflu', 
#             98:'filler', 99:'AllPunc', 100:'Period', 101:'Comma', 102:'Colon', 103:'SemiC', 104:'QMark', 105:'Exclam', 
#             106:'Dash', 107:'Quote', 108:'Apostro', 109:'Parenth', 110:'OtherP', 
            
#             111:'characters', 112:'complexWords', 113:'longWords', 114:'numberSyllables', 115:'lexicon_count', 
#             116:'sentence_count', 117:'flesch_reading_ease', 118:'smog_index', 119:'flesch_kincaid_grade', 
#             120:'coleman_liau_index', 121:'automated_readability_index', 122:'difficult_words', 
#             123:'linsear_write_formula', 124:'gunning_fog', 125:'word_count', 126:'words_per_sent', 
#             127:'nr_captalized_words', 128:'nr_per_stopwords', 129:'count_urls', 
            
#             130:'conjunction, coordinating', 131:'numeral, cardinal', 132:'determiner', 133:'foreign word', 
#             134:'preposition or conjunction, subordinating', 135:'adjective or numeral, ordinal', 
#             136:'adjective, comparative', 137:'adjective, superlative', 138:'modal auxiliary', 
#             139:'noun, common, singular or mass', 140:'noun, proper, singular', 141:'noun, proper, plural', 
#             142:'noun, common, plural', 143:'pre-determiner', 144:'genitive marker', 145:'pronoun, personal', 
#             146:'pronoun, possessive', 147:'adverb', 148:'adverb, comparative', 149:'adverb, superlative', 
#             150:'particle', 151:'"to" as preposition/infinitive', 152:'interjection', 153:'verb, base form', 
#             154:'verb, past tense', 155:'verb, present participle or gerund', 156:'verb, past participle', 
#             157:'verb, present tense, not 3rd person singular', 158:'verb, present tense, 3rd person singular', 
#             159:'WH-determiner', 160:'WH-pronoun', 161:'WH-pronoun, possessive', 162:'Wh-adverb'}

In [8]:

def filter_features_by_index(features, filter_indexes):
    features_temp = []
    for i in range(len(features)):
        if i not in filter_indexes:
            features_temp.append(features[i])
    features_filtered = features_temp
    return features_filtered

# features = [4,5,7,17,22,25,26,29,32,33] #17, 22, 26
# removee = [3,4, 6]
# arraydd = filter_features(features, removee)
# print(arraydd)
# print(filter_features(features, removee))
        

### Select features (Web-markup, LIWIC, Readability and NLTK features)

 <b>Function: </b> extract_features_filter(typefeatures, type_text, filename, label, filter_features). This function selects some of the features: Web-markup, LIWIC, Readability and NLTK features. A previous process was done to identify the relevant features (see the published paper). Here we are just getting these preselected features. This fuction do not need to process any features, they SHOULD be already in the csv file. It was done to get the results more quickly. <br />
 <br />
 <b>Parameters:</b> <br />
 <b>typefeatures</b>: It is an array of strings. The values in the array can be: 'linguisticF_NLTK' for NLTK features, 'linguisticF_LIWC' for LIWIC features, 'readabilityF' for readability features, 'webmarkupF' for web-markup features. All the values that are included in the array will be used as features to build the model. <br />
 <br />
 <b>type_text:</b> It defines the column index - in the csv file - of the text (headline, content or headline+content) that was used to extract LIWIC, readability and NLTK features. Here, this parameter is used just to filter those news with non-text. This parameter can take the values: 1 or 2. 1 = Headline, 2 = Content, 2 = HeadlineContent. Content and HeadlineContent have the same value (2) because they are saved in the same position (third column) in the csv file. This column can contains the content text or headline+content text (string concatenation). 
 <br />         
 <b>filename</b>: csv file. This csv file should contains already the following features: urls(1), headline(1), content or headline+content(1), date(1), Webmarkup (14 features), LIWC (93 features), Readability (19 features) and NLKT (33 features).
 <br />         
 <b>label:</b>It is a numerical value. it could be 1 or 0. 1 means fake news and 0 means real news.
 <br />         
 <b>filter_features:</b> It is an array of integers. The array contains the feature's indexes of the ones that will be filtered. Those features were selected after some additional feature engeeniering      

In [9]:
def extract_features_filter(typefeatures, type_text, filename, label, filter_features ):
    list_docs, list_labels, list_urls = [], [], []
    count_urls, count_mentions = 0, 0
#     print(filter_features)
    filter_liwc = filter_features[0]
    filter_readability = filter_features[1]
    filter_nltk = filter_features[2]
    
    #print ("READY")
    with open(filename, "r") as arq_in:
        reader_in = reader(arq_in, delimiter=',', quoting=QUOTE_ALL)
        for t in reader_in:
            
            if(len(t[0]) > 1 and len(t[type_text]) > 1):#handling missing features

                #get html features
                author = t[4]
                freq_images, freq_audio_video, freq_links, freq_ads  = 0,0,0,0
                if(len(t[9]) > 1):
                    freq_images = t[9]
                if(len(t[10]) > 1):
                    freq_audio_video = t[10]
                if(len(t[11]) > 1):
                    freq_links = t[11]
                if(len(t[17]) > 1):
                    freq_ads = t[17]
                    
                if(len(t[12]) > 1):
                    freq_lists = t[12]
                if(len(t[14]) > 1):
                    freq_styles_semantics = t[14]
                if(len(t[5]) > 1):
                    freq_basic = t[5]
                    
                #Adding all features   
                item = []
                if ('linguisticF_NLTK' in typefeatures):
                    linguistic_nltk = t[130:163]
                    linguistic_nltk_filtered = filter_features_by_index(linguistic_nltk, filter_nltk)
                    item = item + list(linguistic_nltk_filtered) 
                    
                if ('linguisticF_LIWC' in typefeatures):
                    linguistic_liwc = t[18:111]
                    linguistic_liwc_filtered = filter_features_by_index(linguistic_liwc, filter_liwc)
                    item = item + list(linguistic_liwc_filtered)

                if ('readabilityF' in typefeatures):
                    readability_f = t[111:130]
                    readability_f_filtered = filter_features_by_index(readability_f, filter_readability)
                    item = item + list(readability_f_filtered)
                               
                if ('webmarkupF' in typefeatures):
                    item = item + [author, freq_images, freq_links, freq_styles_semantics, freq_basic, freq_ads]
                    
                list_docs.append(item)
                list_labels.append(label)
                list_urls.append(t[0])
    #print ("Finish")
    arq_in.close()
    return list_docs,list_labels, list_urls

In [7]:
#This takes into account the new Web-markup features (it was not used in the WWW paper)
#Future work (Zhohan)
def extract_features_filter_newWM(typefeatures, type_text, filename, label, filter_features):
    list_docs, list_labels, list_urls = [], [], []
    count_urls, count_mentions = 0, 0
    filter_liwc = filter_features[0]
    filter_readability = filter_features[1]
    filter_nltk = filter_features[2]
    
    #print ("READY")
    with open(filename, "r") as arq_in:
        reader_in = reader(arq_in, delimiter=',', quoting=QUOTE_ALL)
        for t in reader_in:
            if ('webmarkupF_new' in typefeatures):
                # return all webmarkup features (t[0] is url), t[-19] is div
                item = t[1:]
                list_docs.append(item)
                list_labels.append(label)
                list_urls.append(t[0])
                continue
            
            if(len(t[0]) > 1 and len(t[type_text]) > 1):#handling missing features

                #get html features
                author = t[4]
                freq_images, freq_audio_video, freq_links, freq_ads  = 0,0,0,0
                if(len(t[9]) > 1):
                    freq_images = t[9]
                if(len(t[10]) > 1):
                    freq_audio_video = t[10]
                if(len(t[11]) > 1):
                    freq_links = t[11]
                if(len(t[17]) > 1):
                    freq_ads = t[17]
                    
                if(len(t[12]) > 1):
                    freq_lists = t[12]
                if(len(t[14]) > 1):
                    freq_styles_semantics = t[14]
                if(len(t[5]) > 1):
                    freq_basic = t[5]
                    
                #Adding all features   
                item = []
                if ('linguisticF_NLTK' in typefeatures):

                    linguistic_nltk = t[130:163]
                    linguistic_nltk_filtered = filter_features_by_index(linguistic_nltk, filter_nltk)
                    if to_print:
                        print('has linguisticF_NLTK')
                        pprint(linguistic_nltk)
                        print(linguistic_nltk_filtered)
                    item = item + list(linguistic_nltk_filtered) 
                    
                if ('linguisticF_LIWC' in typefeatures):
#                     print('has linguisticF_LIWC')
                    linguistic_liwc = t[18:111]
                    linguistic_liwc_filtered = filter_features_by_index(linguistic_liwc, filter_liwc)
                    item = item + list(linguistic_liwc_filtered)

                if ('readabilityF' in typefeatures):
#                     print('has readabilityF')
                    readability_f = t[111:130]
                    readability_f_filtered = filter_features_by_index(readability_f, filter_readability)
                    if to_print:
                        print('has readabilityF')
                        pprint(readability_f)
                        print(readability_f_filtered)
                    item = item + list(readability_f_filtered)
                               
                if ('webmarkupF' in typefeatures):
#                     print('has webmarkupF')
#                     print('WARNING: author is not used!!!')
#                     item = item + [freq_images, freq_links, freq_styles_semantics, freq_basic, freq_ads]
                    item = item + [author, freq_images, freq_links, freq_styles_semantics, freq_basic, freq_ads]
    
                list_docs.append(item)
                list_labels.append(label)
                list_urls.append(t[0])
    #print ("Finish")
    arq_in.close()
    return list_docs,list_labels, list_urls

In [8]:
# CLASSIFIER

#Applying classifier over the test dataset using invariant features
def test_classifier(dir_file_test_fake, dir_file_test_msm, classifier, typefeatures, type_text, filter_features ):
    X_fakenews, fakenews_labels_list, fakenews_list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_fake, 1, filter_features ) #1 means fake news. 
    X_msmnews, msmnews_labels_list, msmnews_list_urls = extract_features_filter(typefeatures, type_text, dir_file_test_msm, 0, filter_features )#0 means true news.
    X = np.asarray(X_fakenews+ X_msmnews)
    #print (X)
    y_true = np.asarray(fakenews_labels_list + msmnews_labels_list) 
    # y_pred = classifier.predict(X) 
    return X, y_true

#Using CrossValidation
def run_classifiers(dir_file_test_fake, dir_file_test_msm, svm_classifier, features, type_text, filter_features ):
    metric = [] #accuracy
    for i in range(len(features)):
        typefeatures = features[i]
        name_features = '-'.join(typefeatures)
        X, y_target = test_classifier(dir_file_test_fake, dir_file_test_msm, svm_classifier, typefeatures, type_text, filter_features)
        scores = cross_val_score(svm_classifier, X, y_target, cv=5)
        print("Accuracy based on " +name_features+ ": %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
        #print(scores)
        metric.append(scores.mean())
    return metric

In [8]:
#Feature that will be filtered. The arrays contain the feature's indexes. Those features were selected after feature engeeniering
#HEADLINES
h_liwc_filter = [85, 55, 74, 77]
h_readability_filter = [7, 18]
h_nlkt_filter = [3, 4, 6, 16, 21, 24, 25, 28, 31, 32]
headline_filter_features = [h_liwc_filter, h_readability_filter, h_nlkt_filter]

#CONTENT
c_liwc_filter = [23, 24, 25, 26, 27, 28]
c_readability_filter = []
c_nlkt_filter = [2, 13, 18, 20, 22]
content_filter_features = [c_liwc_filter, c_readability_filter, c_nlkt_filter]

#HEAD_CONTENT
hc_liwc_filter = [23, 24, 25, 26, 27, 28]
hc_readability_filter = []
hc_nlkt_filter = [2, 7, 13, 14, 18, 19, 22, 32]
headline_content_filter_features = [hc_liwc_filter, hc_readability_filter, hc_nlkt_filter]

In [10]:
path_celebrity = "/Users/.../Fakenews/paper_fn/Experiments/data/celebrity_LIWC/"

fake_files = ['LIWC2015_fakenews_celebrity.csv','LIWC2015_fakenews_celebrity_content.csv','LIWC2015_fakenews_celebrity_headContent.csv']
msm_files = ['LIWC2015_realnews_celebrity.csv', 'LIWC2015_realnews_celebrity_content.csv', 'LIWC2015_realnews_celebrity_headContent.csv']

data = [fake_files, msm_files]
dataLIWC = data
    
svm_classifier = svm.SVC( C=3)#kernel = 'linear'

In [11]:
# Based on headline
dir_file_test_fake =path_celebrity+ dataLIWC[0][0]
dir_file_test_msm =path_celebrity+ dataLIWC[1][0]
features = [['linguisticF_NLTK'], ['linguisticF_LIWC'], ['readabilityF'], ['webmarkupF'],
            ['linguisticF_NLTK', 'linguisticF_LIWC'], ['linguisticF_NLTK', 'readabilityF'], ['linguisticF_NLTK', 'webmarkupF'],
            ['linguisticF_LIWC', 'readabilityF'], ['linguisticF_LIWC', 'webmarkupF'],
            ['readabilityF', 'webmarkupF'],
            ['linguisticF_NLTK', 'linguisticF_LIWC', 'readabilityF'], ['linguisticF_NLTK', 'linguisticF_LIWC', 'webmarkupF'], ['linguisticF_NLTK', 'readabilityF', 'webmarkupF'],
            ['linguisticF_LIWC', 'readabilityF', 'webmarkupF'],
            ['linguisticF_NLTK', 'linguisticF_LIWC', 'readabilityF', 'webmarkupF']]
type_text = 1  #1 = headline
#HEADLINES
filter_features = headline_filter_features

total_gram = 0
total_syntax = 0

celebrity_accuracy_headline = run_classifiers(dir_file_test_fake, dir_file_test_msm, svm_classifier, features, type_text, filter_features)

Accuracy based on linguisticF_NLTK: 0.59 (+/- 0.03)
Accuracy based on linguisticF_LIWC: 0.50 (+/- 0.00)
Accuracy based on readabilityF: 0.49 (+/- 0.09)
Accuracy based on webmarkupF: 0.68 (+/- 0.09)
Accuracy based on linguisticF_NLTK-linguisticF_LIWC: 0.50 (+/- 0.00)
Accuracy based on linguisticF_NLTK-readabilityF: 0.53 (+/- 0.14)
Accuracy based on linguisticF_NLTK-webmarkupF: 0.73 (+/- 0.05)
Accuracy based on linguisticF_LIWC-readabilityF: 0.50 (+/- 0.00)
Accuracy based on linguisticF_LIWC-webmarkupF: 0.50 (+/- 0.00)
Accuracy based on readabilityF-webmarkupF: 0.57 (+/- 0.08)
Accuracy based on linguisticF_NLTK-linguisticF_LIWC-readabilityF: 0.50 (+/- 0.00)
Accuracy based on linguisticF_NLTK-linguisticF_LIWC-webmarkupF: 0.50 (+/- 0.00)
Accuracy based on linguisticF_NLTK-readabilityF-webmarkupF: 0.62 (+/- 0.09)
Accuracy based on linguisticF_LIWC-readabilityF-webmarkupF: 0.50 (+/- 0.00)
Accuracy based on linguisticF_NLTK-linguisticF_LIWC-readabilityF-webmarkupF: 0.50 (+/- 0.00)
